In [ ]:
import random
import numpy
import itertools
import math

from deap import base
from deap import creator
from deap import tools
from deap.algorithms import eaSimple

In [ ]:
# Helper functions

# Create an iterator to iterate over another iterator in chunks
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

In [ ]:
num_people = 5
num_tables = 2
num_seats_per_table = 3
preferred_together = [
    [ 1, 2 ],
    [ ],
    [ 0 ],
    [ ],
    [ ]
]
preferred_apart = [
    [ 0 ],
    [ 3 ],
    [ ],
    [ 1 ],
    [ 0 ]
]

num_empty_seats = num_tables * num_seats_per_table - num_people
permutation_population = list(range(num_people)) + [None] * num_empty_seats

permutation_population

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("permutation", random.sample, permutation_population, len(permutation_population))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
def evaluate(num_seats_per_table, preferred_together, preferred_apart, individual):
    penalty = 0
    num_people = len(preferred_together)
    tables = grouper(individual, num_seats_per_table)
    
    for table in tables:
        for person in table:
            if person is None:
                continue
            
            # Check that person is not alone when they should be with at
            # least one other person
            should_not_be_alone = False
            is_alone = True
            for other_person in preferred_together[person]:
                should_not_be_alone = True
                if other_person in table:
                    is_alone = False
                    break
                    
            if should_not_be_alone and is_alone:
                penalty += 1
                
            # Only consider the people whose id is the lesser of the pair
            # so that each symmetrical preference is calculated only once
            for other_person in preferred_together[person]:
                if other_person < person:
                    continue
                if other_person not in table:
                    penalty += 1
                    
            for other_person in preferred_apart[person]:
                if other_person < person:
                    continue
                if other_person in table:
                    penalty += 1
            
            
    return penalty,

toolbox.register("evaluate", evaluate, num_seats_per_table, preferred_together, preferred_apart)

In [ ]:
# TODO choose a suitable mate function or implement a custom function
toolbox.register("mate", tools.cxTwoPoint)

In [ ]:
# TODO choose a suitable mutate function or implement a custom function
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)

In [ ]:
# TODO choose a suitable select function or implement a custom function
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
# Starting population
seed_population = toolbox.population()
seed_population

In [ ]:
seed_population_size = 10
mate_probability = 0.5
mutate_probability = 0.2
number_generations = 40

def main():
    random.seed(10)
    pop = toolbox.population(n=seed_population_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    eaSimple(pop, toolbox, mate_probability, mutate_probability, number_generations, stats, halloffame=hof)

    return pop, stats, hof

main()

In [ ]:
def test_evaluate():
    num_people = 5
    num_tables = 2
    num_seats_per_table = 3
    preferred_together = [
        [ 1, 2 ],
        [ 0 ],
        [ 0 ],
        [ ],
        [ ]
    ]
    preferred_apart = [
        [ ],
        [ 3 ],
        [ ],
        [ 1 ],
        [ ]
    ]
    
    individual = [0, 1, 2, 3, 4, None]
    assert(evaluate(num_seats_per_table, preferred_together, preferred_apart, individual) == (0,))
    
    individual = [0, 1, 3, 2, 4, None]
    assert(evaluate(num_seats_per_table, preferred_together, preferred_apart, individual) == (3,))

test_evaluate()